機械学習特論 第6回課題 決定木、ランダムフォレスト、アダブーストで アヤメのデータ、MNIST, Fashion-MNIST の分類を比較

In [ ]:
# google colab で実行する場合は、次の行の先頭の # を削除してこのブロックを実行する
#!pip install japanize-matplotlib

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn import metrics
from sklearn.datasets import load_iris
from sklearn.datasets import fetch_openml
from sklearn.neighbors import KNeighborsClassifier
from sklearn import svm
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import HistGradientBoostingClassifier

In [2]:
# sklearnデータセットに収録されたiris(アヤメ)のデータセットをロードしてデータフレームを作成
def load_iris_data():
    data = load_iris()
    x = pd.DataFrame(data["data"],columns=data["feature_names"])
    y = pd.DataFrame(data["target"],columns=["target"])
    return x, y

# 手書き文字のデータセットをダウンロードして、実験用データを準備 (70000枚のうち7000枚を利用)
def load_mnist_data():
    data = fetch_openml('mnist_784', parser='auto', version=1)
    _x = np.array(data['data'].astype(np.float32))
    _y = np.array(data['target'].astype(np.int32))
    _, x, _, y = train_test_split(_x, _y, test_size=0.1, random_state=1, stratify=_y)
    return x, y

# Fashion-MNISTデータセットをダウンロードして、実験用データを準備 (70000枚のうち7000枚を利用)
def load_fashion_mnist_data():
    data = fetch_openml('Fashion-MNIST', parser='auto')
    _x = np.array(data['data'].astype(np.float32))
    _y = np.array(data['target'].astype(np.int32))
    _, x, _, y = train_test_split(_x, _y, test_size=0.1, random_state=1, stratify=_y) 
    return x, y

# 一括処理のためにデータセットの辞書を作成
dataset = {'iris': load_iris_data(), 'mnist': load_mnist_data(), 'fashon-mnist': load_fashion_mnist_data()}

In [17]:
# 一括処理のためにモデルの辞書を作成
model = {
    # k近傍法のモデル
    'kNN(k=3)':
    KNeighborsClassifier(n_neighbors=3, # k を指定 (デフォルトは 5)
                         weights='uniform',  # 距離を考慮しない(uniform:デフォルト)、する(distance)
                         algorithm='auto', # 近傍点計算アルゴリズム (auto:デフォルト,ball_tree,kd_tree,brute)
                         leaf_size=30,  # ball_tree,kd_tree指定時のリーフサイズの設定 (デフォルトは 30)
                         p=2),  # 距離計算の次元 (2:デフォルト、1)
    # svm (kernel="linear", C=1.0) のモデル
    'SVC(kernel="linear", C=1)':
    svm.SVC(kernel="linear", C=1, max_iter=100000, verbose=True, random_state=1),
    # svm (kernel="rbf", C=1) のモデル
    'SVC(kernel="rbf", C=1)':
    svm.SVC(kernel="rbf", C=1, max_iter=100000, verbose=True, random_state=1),
    # 決定木
    'DecisionTree(max_depth=10)':
    DecisionTreeClassifier(max_depth=10, # 木の深さの最大
                                random_state=2), # 乱数シード
    # ランダムフォレストのモデル
    'randomforest(max_depth=10, n_estimators=10)':
    RandomForestClassifier(max_depth=10, # 木の深さの最大
                             n_estimators=10, # 木の数
                             random_state=2), # 乱数シード
    # # アダブーストのモデル (実行時間がかかる場合は n_estimators を減らす)
    'Adaboost(DecisionTreeClassifier, dct(max_depth=10), n_estimators=50)':
    AdaBoostClassifier(estimator=DecisionTreeClassifier(max_depth=10, random_state=1), # ベースモデルを指定 (scikit learn のバージョンにより書式が異なります)
                            n_estimators=50, # 木の数
                            random_state=1), # 乱数シード
    'Adaboost(SVC, n_estimators=10)':
    AdaBoostClassifier(estimator=svm.SVC(kernel="rbf", C=1, max_iter=100000, verbose=True, random_state=1), # ベースモデルを指定 (scikit learn のバージョンにより書式が異なります)
                            n_estimators=10, # 木の数
                            random_state=1, # 乱数シード
                            algorithm='SAMME'), # SVCを使うために AdaBoost のアルゴリズムを指定
    # 'Adaboost(dct(max_depth=10), n_estimators=170)':
    # AdaBoostClassifier(estimator=DecisionTreeClassifier(max_depth=10, random_state=1), # ベースモデルを指定 (scikit learn のバージョンにより書式が異なります)
    #                         n_estimators=170, # 木の数
    #                         random_state=1), # 乱数シード
    # 勾配ブースティングのモデル (実行時間がかかる場合は n_estimators を減らすか HistGradientBoostingClassifier に変更)
    # 'GradientBoostingClassifier(max_depth=5, n_estimators=50)':
    # GradientBoostingClassifier(max_depth=5, # 木の深さの最大
    #                            n_estimators=50, # 木の数
    #                            random_state=1), # 乱数シード
    # 'GradientBoostingClassifier(max_depth=5, n_estimators=170)':
    # GradientBoostingClassifier(max_depth=5, # 木の深さの最大
    #                            n_estimators=170, # 木の数
    #                            random_state=1), # 乱数シード
    'HistGradientBoostingClassifier(max_depth=5)':
    HistGradientBoostingClassifier(max_depth=5, # 木の深さの最大
                               random_state=1), # 乱数シード
}

In [18]:
# 辞書に格納したデータセットそれぞれについて性能を確認
for dataset_key in dataset.keys():
    # データを学習用と検証用に分割
    x, y = dataset[dataset_key]
    x_train, x_test, y_train, y_test = \
        train_test_split(x, y, test_size=0.25, random_state=1, stratify=y) # 検証用データに25%を割当て
    print(f'## dataset:{dataset_key} ',
          f'x_train:{len(x_train)} x_test:{len(x_test)} y_train:{len(y_train)} y_test:{len(y_test)}')

    # 辞書に格納したモデルそれぞれについて性能を測定
    for model_key in model.keys():
        # 学習用データを利用してモデルを学習
        clf = model[model_key]
        clf = clf.fit(x_train, np.array(y_train).ravel()) 

        # 学習したモデルの性能(正答率)を学習用データと検証用データで評価
        predict_train = clf.predict(x_train)
        train_score = metrics.accuracy_score(y_train, predict_train)
        predict_test = clf.predict(x_test)
        test_score = metrics.accuracy_score(y_test, predict_test)
        print(f'dataset:{dataset_key} model:{model_key}', 
            f'accuracy_score: train_data:{train_score: 0.5} test_data:{test_score: 0.5}')


## dataset:iris  x_train:112 x_test:38 y_train:112 y_test:38
dataset:iris model:kNN(k=3) accuracy_score: train_data: 0.96429 test_data: 0.97368
[LibSVM]dataset:iris model:SVC(kernel="linear", C=1) accuracy_score: train_data: 0.97321 test_data: 1.0
[LibSVM]dataset:iris model:SVC(kernel="rbf", C=1) accuracy_score: train_data: 0.96429 test_data: 0.97368
dataset:iris model:DecisionTree(max_depth=10) accuracy_score: train_data: 1.0 test_data: 0.97368
dataset:iris model:randomforest(max_depth=10, n_estimators=10) accuracy_score: train_data: 0.99107 test_data: 0.97368
dataset:iris model:Adaboost(DecisionTreeClassifier, dct(max_depth=10), n_estimators=50) accuracy_score: train_data: 1.0 test_data: 0.97368
[LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM]dataset:iris model:Adaboost(SVC, n_estimators=10) accuracy_score: train_data: 0.33929 test_data: 0.31579
dataset:iris model:HistGradientBoostingClassifier(max_depth=5) accuracy_score: train_data: 1.0 test_data: 0.

In [19]:
# 辞書に格納したデータセットそれぞれについて性能を確認
for dataset_key in dataset.keys():
    # データを学習用と検証用に分割
    x, y = dataset[dataset_key]
    x_train, x_test, y_train, y_test = \
        train_test_split(x, y, test_size=0.25, random_state=1, stratify=y) # 検証用データに25%を割当て
    print(f'## dataset:{dataset_key} ',
          f'x_train:{len(x_train)} x_test:{len(x_test)} y_train:{len(y_train)} y_test:{len(y_test)}')

    # データ標準化なしで性能を測定
    print('# no scaling')
    # 辞書に格納したモデルそれぞれについて性能を測定
    for model_key in model.keys():
        # 学習用データを利用してモデルを学習
        clf = model[model_key]
        clf = clf.fit(x_train, np.array(y_train).ravel()) 

        # 学習したモデルの性能(正答率)を学習用データと検証用データで評価
        predict_train = clf.predict(x_train)
        train_score = metrics.accuracy_score(y_train, predict_train)
        predict_test = clf.predict(x_test)
        test_score = metrics.accuracy_score(y_test, predict_test)
        print(f'dataset:{dataset_key} model:{model_key}', 
            f'accuracy_score: train_data:{train_score: 0.5} test_data:{test_score: 0.5}')

    # データを標準化
    print('# with scaling')
    scaler = StandardScaler()
    scaler.fit(x_train)
    x_train = scaler.transform(x_train)
    x_test = scaler.transform(x_test)

    # 辞書に格納したモデルそれぞれについて性能を測定
    for model_key in model.keys():
        # 学習用データを利用してモデルを学習
        clf = model[model_key]
        clf = clf.fit(x_train, np.array(y_train).ravel()) 

        # 学習したモデルの性能(正答率)を学習用データと検証用データで評価
        predict_train = clf.predict(x_train)
        train_score = metrics.accuracy_score(y_train, predict_train)
        predict_test = clf.predict(x_test)
        test_score = metrics.accuracy_score(y_test, predict_test)
        print(f'dataset:{dataset_key} model:{model_key}', 
            f'accuracy_score: train_data:{train_score: 0.5} test_data:{test_score: 0.5}')

## dataset:iris  x_train:112 x_test:38 y_train:112 y_test:38
# no scaling
dataset:iris model:kNN(k=3) accuracy_score: train_data: 0.96429 test_data: 0.97368
[LibSVM]dataset:iris model:SVC(kernel="linear", C=1) accuracy_score: train_data: 0.97321 test_data: 1.0
[LibSVM]dataset:iris model:SVC(kernel="rbf", C=1) accuracy_score: train_data: 0.96429 test_data: 0.97368
dataset:iris model:DecisionTree(max_depth=10) accuracy_score: train_data: 1.0 test_data: 0.97368
dataset:iris model:randomforest(max_depth=10, n_estimators=10) accuracy_score: train_data: 0.99107 test_data: 0.97368
dataset:iris model:Adaboost(DecisionTreeClassifier, dct(max_depth=10), n_estimators=50) accuracy_score: train_data: 1.0 test_data: 0.97368
[LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM]dataset:iris model:Adaboost(SVC, n_estimators=10) accuracy_score: train_data: 0.33929 test_data: 0.31579
dataset:iris model:HistGradientBoostingClassifier(max_depth=5) accuracy_score: train_data: 1.0 